<div style="padding:20px;color:white;margin:0;font-size:300%;font-weight:800;text-align:center;display:fill;border-radius:60px;background-color:#CC99FF">Filght Price Prediction</div>

# Project. Filght Price Prediction
## Goal
> ## Modeling : Predicting Filght Price 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flight-price-data/flight_dataset.csv


# Import Libraries & Read Dataset

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
# For EDA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# For machine learning
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# For Deep learning
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN, LSTM , Input , Embedding

In [4]:
df = pd.read_csv('/kaggle/input/flight-price-data/flight_dataset.csv')
df

,Airline,Source,Destination,Total_Stops,Price,Date,Month,Year,Dep_hours,Dep_min,Arrival_hours,Arrival_min,Duration_hours,Duration_min
0,IndiGo,Banglore,New Delhi,0,3897,24,3,2019,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,2,7662,1,5,2019,5,50,13,15,7,25
2,Jet Airways,Delhi,Cochin,2,13882,9,6,2019,9,25,4,25,19,0
3,IndiGo,Kolkata,Banglore,1,6218,12,5,2019,18,5,23,30,5,25
4,IndiGo,Banglore,New Delhi,1,13302,1,3,2019,16,50,21,35,4,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,0,4107,9,4,2019,19,55,22,25,2,30
10679,Air India,Kolkata,Banglore,0,4145,27,4,2019,20,45,23,20,2,35
10680,Jet Airways,Banglore,Delhi,0,7229,27,4,2019,8,20,11,20,3,0
10681,Vistara,Banglore,New Delhi,0,12648,1,3,2019,11,30,14,10,2,40


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Airline         10683 non-null  object
 1   Source          10683 non-null  object
 2   Destination     10683 non-null  object
 3   Total_Stops     10683 non-null  int64 
 4   Price           10683 non-null  int64 
 5   Date            10683 non-null  int64 
 6   Month           10683 non-null  int64 
 7   Year            10683 non-null  int64 
 8   Dep_hours       10683 non-null  int64 
 9   Dep_min         10683 non-null  int64 
 10  Arrival_hours   10683 non-null  int64 
 11  Arrival_min     10683 non-null  int64 
 12  Duration_hours  10683 non-null  int64 
 13  Duration_min    10683 non-null  int64 
dtypes: int64(11), object(3)
memory usage: 1.1+ MB


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Total_Stops,10683.0,0.824207,0.675199,0.0,0.0,1.0,1.0,4.0
Price,10683.0,9087.064121,4611.359167,1759.0,5277.0,8372.0,12373.0,79512.0
Date,10683.0,13.508378,8.479277,1.0,6.0,12.0,21.0,27.0
Month,10683.0,4.708602,1.164357,3.0,3.0,5.0,6.0,6.0
Year,10683.0,2019.000000,0.000000,2019.0,2019.0,2019.0,2019.0,2019.0
Dep_hours,10683.0,12.490686,5.748650,0.0,8.0,11.0,18.0,23.0
Dep_min,10683.0,24.411214,18.767980,0.0,5.0,25.0,40.0,55.0
Arrival_hours,10683.0,13.348778,6.859125,0.0,8.0,14.0,19.0,23.0
Arrival_min,10683.0,24.690630,16.506036,0.0,10.0,25.0,35.0,55.0
Duration_hours,10683.0,10.246560,8.494988,1.0,2.0,8.0,15.0,47.0


# Column Analysis (numerical)

In [7]:
col_name_object = df.select_dtypes(include='object').columns.tolist()
col_name_num = df.select_dtypes(include=['float64','int64']).columns.tolist()
time_series = ['Year', 'Month','Date']

for col in col_name_object:
    print(f"Column '{col}' unquie values: {df[col].unique()}")
    
for col in col_name_num:
    print(f"Column '{col}' unquie values: {df[col].unique()}")

Column 'Airline' unquie values: ['IndiGo' 'Air India' 'Jet Airways' 'SpiceJet' 'Multiple carriers' 'GoAir'
 'Vistara' 'Air Asia' 'Vistara Premium economy' 'Jet Airways Business'
 'Multiple carriers Premium economy' 'Trujet']
Column 'Source' unquie values: ['Banglore' 'Kolkata' 'Delhi' 'Chennai' 'Mumbai']
Column 'Destination' unquie values: ['New Delhi' 'Banglore' 'Cochin' 'Kolkata' 'Delhi' 'Hyderabad']
Column 'Total_Stops' unquie values: [0 2 1 3 4]
Column 'Price' unquie values: [ 3897  7662 13882 ...  9790 12352 12648]
Column 'Date' unquie values: [24  1  9 12 27 18  3 15  6 21]
Column 'Month' unquie values: [3 5 6 4]
Column 'Year' unquie values: [2019]
Column 'Dep_hours' unquie values: [22  5  9 18 16  8 11 20 21 17 14  4  7 10 15  6 19 23 13  2 12  0  1  3]
Column 'Dep_min' unquie values: [20 50 25  5  0 55 45 40 10 15 35 30]
Column 'Arrival_hours' unquie values: [ 1 13  4 23 21 11 10  5 19 22  9 12  6  8  3 20 16 18 15 14  0  2  7 17]
Column 'Arrival_min' unquie values: [10 15 25 3

In [8]:
fig = go.Figure()

col_name_num = [element for element in col_name_num if element not in time_series]


for col in col_name_num:
    fig = go.Figure()
    fig.add_trace(go.Box(y=df[col], name=col))

    fig.update_layout(
        title=f"Box Plot for {col}",
        yaxis_title="Values",
        title_font_size=25,
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )

    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='white', griddash='dash')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='white', griddash='dash')

    fig.show()

# Column Analysis (time series)

In [9]:
time_series.remove('Year')

df_monthly = df.groupby('Month')['Price'].sum().reset_index()
df_date = df.groupby('Date')['Price'].sum().reset_index()


for time in time_series:
    if time == 'Month':
        data = df_monthly
        x_label = 'Month'
    elif time == 'Date':
        data = df_date
        x_label = 'Date'
        
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=data[x_label],
        y=data['Price'],
        mode='lines',
        name=f"Price vs {time}",
        line=dict(color='red')
    ))

    fig.update_layout(
        title=f"Time Series Plot for Price vs {time}",
        xaxis_title=x_label,
        yaxis_title='Total Price',
        title_font_size=25,
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )

    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='white', griddash='dash')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='white', griddash='dash')

    fig.show()

# Machine learning

In [10]:
def evaluate_models_all(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    categorical_features = col_name_object
    models = [
        ('Linear Regression', LinearRegression()),
        ('Ridge Regression', Ridge()),
        ('Lasso Regression', Lasso()),
        ('ElasticNet', ElasticNet()),
        ('Decision Tree Regressor', DecisionTreeRegressor()),
        ('Random Forest Regressor', RandomForestRegressor()),
        ('Support Vector Machine Regressor', SVR()),
        ('K-Nearest Neighbors Regressor', KNeighborsRegressor()),
        ('Gradient Boosting Regressor', GradientBoostingRegressor()),
        ('XGBoost Regressor', XGBRegressor()),
        ('LightGBM Regressor', LGBMRegressor())
    ]

    pipelines = {}
    for name, model in models:
        preprocessor = ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(drop='first'), categorical_features)
            ],
            remainder='passthrough'
        )

        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('scaler', StandardScaler()),
            (name, model)
        ])

        pipelines[name] = pipeline

    results = {}
    trained_models = {}
    for name, pipeline in pipelines.items():
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        r2 = r2_score(y_test, y_pred)
        results[name] = {'RMSE': rmse, 'R²': r2}
        trained_models[name] = pipeline.named_steps[name]

    results_df = pd.DataFrame.from_dict(results, orient='index').reset_index()
    results_df.rename(columns={'index': 'Model'}, inplace=True)

    return results_df

In [11]:
X = df.drop('Price', axis=1)
y = df['Price']

evaluate_models_all(X,y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002841 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 200
[LightGBM] [Info] Number of data points in the train set: 8546, number of used features: 25
[LightGBM] [Info] Start training from score 9074.827873


,Model,RMSE,R²
0,Linear Regression,2855.868946,0.614629
1,Ridge Regression,2854.694163,0.614946
2,Lasso Regression,2854.729643,0.614936
3,ElasticNet,3004.505615,0.573471
4,Decision Tree Regressor,2490.551855,0.706915
5,Random Forest Regressor,2133.279593,0.784970
6,Support Vector Machine Regressor,4475.604197,0.053531
7,K-Nearest Neighbors Regressor,2210.296098,0.769164
8,Gradient Boosting Regressor,2198.302366,0.771662
9,XGBoost Regressor,1735.964124,0.857608


# Deep learning 

> ## 1. MLP (Multi-Layer Perceptron)

In [12]:
label_encoders = {}
for col in col_name_object:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model_MLP = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1)  
])

model_MLP.compile(optimizer='adam', loss='mean_squared_error')  
model_MLP.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
102/268 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 104702776.0000

I0000 00:00:1719377586.355001      81 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1719377586.372703      81 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 103462152.0000

W0000 00:00:1719377589.886404      81 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


268/268 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - loss: 103459760.0000 - val_loss: 101072488.0000
Epoch 2/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 93545456.0000 - val_loss: 72585968.0000
Epoch 3/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 63104956.0000 - val_loss: 32820268.0000
Epoch 4/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29585628.0000 - val_loss: 15531147.0000
Epoch 5/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17340318.0000 - val_loss: 8594319.0000
Epoch 6/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 13441058.0000 - val_loss: 5306029.0000
Epoch 7/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11248192.0000 - val_loss: 3704819.0000
Epoch 8/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 10230073.0000 - val_loss: 2785774.0000
Epoch 9/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9953738.0000 - val_loss: 2444532.5000
Epoch 10/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 8585677.0000 - val_loss: 1876398.1250
Epoch 11/50
268

In [13]:
mlp_y_pred = model_MLP.predict(X_test)
mlp_rmse = mean_squared_error(y_test, mlp_y_pred, squared=False)
mlp_r2 = r2_score(y_test, mlp_y_pred)

print(f'RMSE: {mlp_rmse:.4f}')
print(f'R²: {mlp_r2:.4f}')

43/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step   

W0000 00:00:1719377616.779998      81 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
RMSE: 707.4161
R²: 0.9764


> ## 2. RNN (Recurrent Neural Network)

In [14]:
X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))  
X_test_rnn = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

model_RNN = Sequential([
    Input(shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),  
    SimpleRNN(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1)  
])

model_RNN.compile(optimizer='adam', loss='mean_squared_error')  
model_RNN.fit(X_train_rnn, y_train, epochs=50, batch_size=32, validation_data=(X_test_rnn, y_test))

Epoch 1/50
 91/268 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 107913088.0000

W0000 00:00:1719377622.264001      82 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 106510368.0000

W0000 00:00:1719377626.366271      81 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


268/268 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - loss: 106496336.0000 - val_loss: 100630032.0000
Epoch 2/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 96722712.0000 - val_loss: 71506856.0000
Epoch 3/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 59364824.0000 - val_loss: 31211340.0000
Epoch 4/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 28781774.0000 - val_loss: 14374244.0000
Epoch 5/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 17352982.0000 - val_loss: 8243001.5000
Epoch 6/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 13295120.0000 - val_loss: 4992570.0000
Epoch 7/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 11513951.0000 - val_loss: 3594426.5000
Epoch 8/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 9810678.0000 - val_loss: 2750897.0000
Epoch 9/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 9997139.0000 - val_loss: 2106287.0000
Epoch 10/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 9285001.0000 - val_loss: 1843916.1250
Epoch 11/50
268/

In [15]:
rnn_y_pred = model_RNN.predict(X_test_rnn)
rnn_rmse = mean_squared_error(y_test, rnn_y_pred, squared=False)
rnn_r2 = r2_score(y_test, rnn_y_pred)

print(f'RMSE: {rnn_rmse:.4f}')
print(f'R²: {rnn_r2:.4f}')

39/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step   

W0000 00:00:1719377656.837871      79 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
RMSE: 710.0019
R²: 0.9762


> ## 3. LSTM (Long Short-Term Memory)

In [16]:
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1])) 
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

model_LSTM = Sequential([
    Input(shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),  
    LSTM(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1)  
])

model_LSTM.compile(optimizer='adam', loss='mean_squared_error')  
model_LSTM.fit(X_train_lstm, y_train, epochs=50, batch_size=32, validation_data=(X_test_lstm, y_test))

Epoch 1/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - loss: 103330264.0000 - val_loss: 100557952.0000
Epoch 2/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 93319000.0000 - val_loss: 63040432.0000
Epoch 3/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 52196948.0000 - val_loss: 24445398.0000
Epoch 4/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 23313124.0000 - val_loss: 12234235.0000
Epoch 5/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15767514.0000 - val_loss: 7506256.5000
Epoch 6/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 12491794.0000 - val_loss: 5240673.0000
Epoch 7/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 11241796.0000 - val_loss: 3875835.0000
Epoch 8/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 9526123.0000 - val_loss: 2876196.2500
Epoch 9/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 9298680.0000 - val_loss: 2304086.2500
Epoch 10/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 9118310.0000 - val_loss: 1871019.3750
Epoch

In [17]:
lstm_y_pred = model_LSTM.predict(X_test_lstm)
lstm_rmse = mean_squared_error(y_test, lstm_y_pred, squared=False)
lstm_r2 = r2_score(y_test, lstm_y_pred)  

print(f'RMSE: {lstm_rmse:.4f}')
print(f'R²: {lstm_r2:.4f}')

67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
RMSE: 585.7038
R²: 0.9838


# Conclusion

In [18]:
print('\nMachine Learning\n')
evaluate_models_all(X,y)


Machine Learning

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002563 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 200
[LightGBM] [Info] Number of data points in the train set: 8546, number of used features: 25
[LightGBM] [Info] Start training from score 9074.827873


,Model,RMSE,R²
0,Linear Regression,2855.868946,0.614629
1,Ridge Regression,2854.694163,0.614946
2,Lasso Regression,2854.729643,0.614936
3,ElasticNet,3004.505615,0.573471
4,Decision Tree Regressor,2496.175476,0.705590
5,Random Forest Regressor,2128.568050,0.785919
6,Support Vector Machine Regressor,4475.604197,0.053531
7,K-Nearest Neighbors Regressor,2210.296098,0.769164
8,Gradient Boosting Regressor,2198.302366,0.771662
9,XGBoost Regressor,1735.964124,0.857608


In [19]:
results_df = pd.DataFrame({
    'Model': ['Multi-Layer Perceptron', 'Recurrent Neural Network', 'Long Short-Term Memory'],
    'RMSE': [mlp_rmse, rnn_rmse, lstm_rmse],
    'R²': [mlp_r2, rnn_r2, lstm_r2]
})
print('\nDeep Learning\n')
results_df


Deep Learning



,Model,RMSE,R²
0,Multi-Layer Perceptron,707.416074,0.976354
1,Recurrent Neural Network,710.001866,0.976181
2,Long Short-Term Memory,585.703819,0.983791


# Thanks